## Predication with Different Classification Method to The Dataset

In [247]:
import pandas as pd
import numpy as np

from sklearn.exceptions import ConvergenceWarning
import warnings

from sklearn.ensemble import IsolationForest

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers import Dense

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier

# Ensemble Methods
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import os

In [248]:
# Suppress specific warning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

warnings.filterwarnings("ignore")

os.environ["PYTHONWARNINGS"] = "ignore"

In [249]:
# Set max output lines before scrolling
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.instance().display_formatter.formatters['text/plain'].for_type(
    type, lambda obj, p, cycle: p.text(repr(obj)[:10000])
)

<function __main__.<lambda>(obj, p, cycle)>

### Metrics Generation

In [250]:
#Metric Calculations

def calculate_metrics(classifier, y_val, y_pred):
    print(f"{classifier} metrics: ")

    print(classification_report(y_val, y_pred))

In [251]:
def train_and_accuracy_gen(model, label, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    calculate_metrics(label, y_test, model.predict(X_test))

In [252]:
class ModelEvaluationPipeline:

    param_grid_logistic_regression = {
        'C': [0.01, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear', 'saga'],
        'penalty': ['l2'],
        'max_iter': [100, 500, 1000]
    }

    param_grid_decission_tree_classifier = {
        'max_depth': [None, 5, 20, 50],
        'min_samples_split': [2, 5, 10, 20],
        'criterion': ['gini', 'entropy'],
    }

    param_grid_random_forest_classifier = {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 20, 50],
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy']
    }

    param_grid_gaussian_naive_bias = {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
    }

    param_grid_svc = {
        'C': [0.1, 1, 10, 100, 1000],
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
        'kernel': ['rbf', 'poly']
    }

    param_grid_knn = {
        'n_neighbors': [100, 500, 700, 900, 1100, 1500],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski'],
        'p': [1, 2]
    }

    param_grid_ada_boost = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.5, 1.0],
        'estimator': [
            DecisionTreeClassifier(max_depth=1),
            DecisionTreeClassifier(max_depth=3)
        ],
    }

    param_grid_xgb = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2], 
        'subsample': [0.6, 0.8, 1.0],     
        'gamma': [0, 0.1, 0.3, 0.5],           
    }

    param_grid_ann = {
        'model__n_neurons': [32, 64, 128],
        'model__activation': ['relu', 'tanh'],
        'epochs': [10, 20],
        'batch_size': [16, 32, 64]
    }

    def __init__(self, file_path, label):
        print(f"\n================= Model and scores for {label} =====================\n")
        
        self.feature_path = file_path
        self.feature_df = self.get_feture()
        self.X, self.y = self.split_feture_and_target()
        self.y = self.map_zero_to_n() # mapping y zero to number of class to make it usable for some modles i.e. xgaboost
        self.X_train, self.X_test, self.y_train, self.y_test = self.get_scale_and_test_train_split()

    # data read and processing section
    def remove_outliear(self, feature_df):
        iso = IsolationForest(contamination=0.01, random_state=42)
        outliers = iso.fit_predict(feature_df)
        data_cleaned = feature_df[outliers == 1]

        return data_cleaned

    def get_feture(self):
        feature_df = pd.read_csv(self.feature_path)
        feature_df = feature_df.iloc[:, 1:] # remove index
        
        return self.remove_outliear(feature_df)

    def split_feture_and_target(self):
        X = self.feature_df.iloc[:, :-1]
        y = self.feature_df.iloc[:, -1]
        
        return X, y

    def get_scale_and_test_train_split(self):
        #Scaling
        scaler = StandardScaler()
        scaled_fature = scaler.fit_transform(self.X)
        
        #test train split
        return train_test_split(scaled_fature, self.y, train_size=.20, random_state=42, stratify=self.y)
    
    def map_zero_to_n(self):
        unique_values = {val: idx for idx, val in enumerate(self.y.unique())}
        y_mapped = self.y.map(unique_values)

        return y_mapped
    
    # Cross validation section 
    def kfold_cross_validation(self, model, n_splits):
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        kfold_score = cross_val_score(model, self.X, self.y, cv=kf)

        print("K-fold cross validaiton scores:", kfold_score)

    def stratified_cross_validation(self, model, n_splits):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        skfold_score = cross_val_score(model, self.X, self.y, cv=skf)

        print("Straified cross validation scores:", skfold_score)

    def cross_validation(self, model, n_splits):
        self.kfold_cross_validation(model, n_splits)
        self.stratified_cross_validation(model, n_splits)

    # Hyper parameter tuning

    def gridSerach(self, estimator, param_grid):
        print("==== Grid Search: =====")

        grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=3, verbose=0)
        grid_search.fit(self.X_train, self.y_train)

        print("Best parameters found: ", grid_search.best_params_)
        print("Best score found: ", grid_search.best_score_)

        return grid_search
    
    def randomSearch(self, estimator, param_grid):
        print("\n==== Random Search: =====")

        random_search = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=500, cv=3, random_state=42)
        random_search.fit(self.X_train, self.y_train)

        print("Best parameters found: ", random_search.best_params_)
        print("Best score found: ", random_search.best_score_)

        return random_search
    
    def hyper_parameter_tuning(self, model, param_grid):
        grid_search = self.gridSerach(model, param_grid)
        random_search = self.randomSearch(model, param_grid)

        return grid_search if grid_search.best_score_ > random_search.best_score_ else random_search
    
    # Models section
    def run_logistic_regression_model(self):
        print("\n=============== 1. Logistic Regression Section: ==================\n")

        tuned_model = self.hyper_parameter_tuning(LogisticRegression(), self.param_grid_logistic_regression)
        lrm = tuned_model.best_estimator_

        train_and_accuracy_gen(lrm, "1. Logistic regression", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(lrm, 10)
    
    def run_decission_tree_classifier_model(self):
        print("\n=================2. Decission Tree Classifier Section: ================\n")

        tuned_model = self.hyper_parameter_tuning(DecisionTreeClassifier(), self.param_grid_decission_tree_classifier)
        dt = tuned_model.best_estimator_

        train_and_accuracy_gen(dt, "2. Decission Tree Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(dt, 10)

    def run_random_forest_classifier_model(self):
        print("\n=================== 3. Random Forest Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(RandomForestClassifier(), self.param_grid_random_forest_classifier)
        rfc = tuned_model.best_estimator_

        train_and_accuracy_gen(rfc, "3.  Random Forest Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(rfc, 10)

    def run_gaussian_naive_bias_classifier_model(self):
        print("\n=================== 4. Gaussian Naive Bias Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(GaussianNB(), self.param_grid_gaussian_naive_bias)
        gnb = tuned_model.best_estimator_

        train_and_accuracy_gen(gnb, "4. Gaussian Naive Bias Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(gnb, 10)


    def run_support_vector_classifier_model(self):
        print("\n=================== 5. Support Vector Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(SVC(), self.param_grid_svc)
        svc = tuned_model.best_estimator_

        train_and_accuracy_gen(svc, "5. Support Vector Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(svc, 10)


    def run_knn_classifier_model(self):
        print("\n=================== 6. K-Nearest Neighbors Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(KNeighborsClassifier(), self.param_grid_knn)
        knn = tuned_model.best_estimator_

        train_and_accuracy_gen(knn, "6. K-Nearest Neighbors", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(knn, 10)

    def run_ada_boost_classifier_model(self):
        print("\n=================== 7. Ada Boost Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(AdaBoostClassifier(), self.param_grid_ada_boost)
        abc = tuned_model.best_estimator_

        train_and_accuracy_gen(abc, "7. Ada Boost Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(abc, 10)

    def run_xg_boost_classifier_model(self):
        print("\n=================== 8. XG Boost Classifier Section: ===================\n")

        tuned_model = self.hyper_parameter_tuning(XGBClassifier(), self.param_grid_xgb)
        xgb = tuned_model.best_estimator_

        train_and_accuracy_gen(xgb, "8. XG Boost Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(xgb, 10)

    @staticmethod
    def build_ann(n_neurons=64, activation='relu'):
        model = Sequential()
        # Input layer
        model.add(Dense(n_neurons, activation=activation, input_shape=(24,)))
        
        model.add(Dense(n_neurons, activation=activation))
        model.add(Dense(n_neurons, activation=activation))
            
        # Output layer (example for binary classification)
        model.add(Dense(units=1, activation='sigmoid'))
        
        model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
        
        return model
    
    def run_ann_model(self):
        print("\n=================== 9. Artificial Neural Net Section: ===================\n")

        model = KerasClassifier(build_fn=self.build_ann, verbose=0)
        
        tuned_model = self.hyper_parameter_tuning(model, self.param_grid_ann)
        ann = tuned_model.best_estimator_

        train_and_accuracy_gen(ann, "9. Artificial Neuralnet", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(ann, 10)

    def driver(self):
        self.run_logistic_regression_model()
        self.run_decission_tree_classifier_model()
        self.run_random_forest_classifier_model()
        self.run_gaussian_naive_bias_classifier_model()
        self.run_support_vector_classifier_model()
        self.run_knn_classifier_model()
        self.run_ada_boost_classifier_model()
        self.run_xg_boost_classifier_model()
        # self.run_ann_model()


In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w100_o25_features.csv", "Window 100 & 25% Overlap")
model_evaluation_pipeline.driver()


================= Model and scores for Window 100 & 50% Overlap =====================


=============== 1. Logistic Regression Section: ==================

==== Grid Search: =====
Best parameters found:  {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score found:  0.30028598976074977

==== Random Search: =====
Best parameters found:  {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 100}
Best score found:  0.30028598976074977
1. Logistic regression metrics: 
              precision    recall  f1-score   support

           0       0.34      0.41      0.37      2113
           1       0.21      0.21      0.21      2267
           2       0.14      0.16      0.15      2174
           3       0.23      0.35      0.27      2224
           4       0.91      0.74      0.81      1721
           5       0.16      0.13      0.14      2222
           6       0.12      0.01      0.02      2133
           7       0.36      0.03      0.05      1649
          

KeyboardInterrupt: 

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w100_o50_features.csv", "Window 100 & 50% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w200_o25_features.csv", "Window 200 & 25% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w200_o50_features.csv", "Window 200 & 50% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w300_o25_features.csv", "Window 300 & 25% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w300_o50_features.csv", "Window 300 & 50% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w400_o25_features.csv", "Window 400 & 25% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w400_o50_features.csv", "Window 400 & 50% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w500_o25_features.csv", "Window 500 & 25% Overlap")
model_evaluation_pipeline.driver()

In [ ]:
model_evaluation_pipeline = ModelEvaluationPipeline("features/w500_o50_features.csv", "Window 500 & 50% Overlap")
model_evaluation_pipeline.driver()